# Compare Lexicons

We compare the streams generated with 

1. controlled lexicons (ours),
2. random baseline streams, and
3. streams generated based on reference lexicons from the literature

based on the repetitiveness of the phoneme features.

In [4]:
from arc import load_phonemes, make_syllables, make_words
from arc import make_lexicons, load_words

import numpy as np 
import random

words = load_words("data_submission/words.json")

In [5]:
from copy import copy
import pandas as pd
from arc.types.base_types import Register
import numpy as np

from arc.core.syllable import LABELS_C, LABELS_V, syllable_from_phonemes
from arc.core.word import Word, word_overlap_matrix

phonemes = load_phonemes()

syll_feature_labels = [LABELS_C, LABELS_V]
syllable_type = "cv"

def to_phoneme(phoneme):
    return phoneme

def to_syllable(syllable):
    if len(syllable) == 3 and not syllable.endswith("ː"):
        syllable_obj = syllable_from_phonemes(phonemes, syllable[:2], syll_feature_labels)
        syllable_obj.id = syllable
        return syllable_obj
    return syllable_from_phonemes(phonemes, syllable, syll_feature_labels)

def to_word(word):
    syllables_list = list(map(to_syllable, word))
    word_id = "".join(s.id for s in syllables_list)
    word_features = list(list(tup) for tup in zip(*[s.info["binary_features"] for s in syllables_list]))
    return Word(id=word_id, info={"binary_features": word_features}, syllables=syllables_list)

def to_lexicon(lexicon):
    word_objs_list = list(map(to_word, lexicon))
    lexicon = Register({w.id:  w for w in word_objs_list})
    lexicon.info.update({"syllable_feature_labels": [LABELS_C, LABELS_V],  "syllable_type": syllable_type})
    overlap = word_overlap_matrix(lexicon)
    lexicon.info["cumulative_feature_repetitiveness"] = np.triu(overlap, 1).sum()
    lexicon.info["max_pairwise_feature_repetitiveness"] = np.triu(overlap, 1).max()
    return lexicon

In [6]:
N_LEXICONS = 21  # number of lexicons per TP mode
N_WORDS_PER_LEXICON = 4  
N_REPS = 5  # how often to randomize the lexicon to build the total stream, 
            # i.e. how long will the streams be in lexicon lengths N_REPS*N_WORDS_PER_LEXICON = n words in the stream
N_STREAMS_PER_INPUT = 5

In [7]:
def print_stream_info(stream):
    print("Stream:", "|".join([syll.id for syll in stream]))
    print("TP mode:", stream.info["stream_tp_mode"])
    print("Lexicon:", stream.info["lexicon"])
    print("Feature PRIs:", stream.info["rhythmicity_indexes"])
    print("")

### ARC Lexicons

In [8]:
from arc import make_streams
from arc import make_lexicons, Register, load_phonemes, load_words

print("Load words...")
words = load_words("data_submission/words.json")

print("Make lexicons...")
controlled_lexicons = make_lexicons(words, n_lexicons=N_LEXICONS, n_words=N_WORDS_PER_LEXICON, control_features=True)

for i, lex in enumerate(controlled_lexicons):
    print(i, lex)

print("")
print("Example Lexicon:", controlled_lexicons[0])
print("Info:", controlled_lexicons[0].info)
print("")


Load words...
Make lexicons...


Increasing allowed overlaps: MAX_PAIRWISE_OVERLAP=1, MAX_CUMULATIVE_OVERLAP=1
Increasing allowed overlaps: MAX_PAIRWISE_OVERLAP=1, MAX_CUMULATIVE_OVERLAP=2


0 hiːboːzyː|kuːnyːfoː|vaːtyːhøː|ʃoːkaːmɛː
1 peːhoːʃaː|kaːlyːfuː|zuːpoːhøː|hiːboːzyː
2 hiːboːzyː|ʃoːmeːɡaː|luːkɛːfoː|vaːdeːhoː
3 hiːboːzyː|vaːtoːhøː|peːhuːʃoː|tyːhoːfaː
4 huːpoːʃøː|tɛːheːvaː|ʃaːbyːhiː|fiːloːɡyː
5 ʃoːmeːɡaː|vaːtoːhuː|moːɡiːʃøː|kuːraːfoː
6 hiːboːzyː|ʃeːhoːpaː|vaːnuːɡiː|zuːpoːhøː
7 vaːnuːɡiː|ʃeːhoːbyː|zuːpoːhøː|heːbiːʃoː
8 ʃiːbeːhøː|huːfiːdoː|hiːtuːfoː|faːroːɡyː
9 kaːlyːfuː|huːfiːdoː|poːʃuːhøː|niːkoːvaː
10 hiːboːzyː|vaːdeːhoː|reːkoːfaː|kuːraːfoː
11 ʃuːhoːbøː|reːfyːkoː|heːdoːfiː|faːɡiːryː
12 foːkaːnuː|hiːbyːʃoː|ɡaːluːfyː|zuːpoːhøː
13 fyːhoːtiː|ʃoːmeːɡaː|huːzyːpoː|kaːfuːreː
14 ɡaːmoːzɛː|huːdoːfaː|reːvaːkɛː|poːʃuːhøː
15 peːhoːʃaː|hiːboːzyː|ʃoːkaːmɛː|kuːnɛːfoː
16 nyːfuːɡiː|hiːtuːfoː|vaːkoːryː|tyːhoːfaː
17 hiːboːzyː|huːdoːfaː|ʃaːmoːɡɛː|poːʃuːhøː
18 beːhoːzɛː|ʃoːmeːɡaː|huːzyːpoː|kaːfuːreː
19 ɡaːmoːzɛː|ruːfyːɡiː|hiːtoːfaː|fiːkaːryː
20 ʃuːhoːpeː|vaːkoːryː|heːpoːzɛː|foːniːkaː

Example Lexicon: hiːboːzyː|kuːnyːfoː|vaːtyːhøː|ʃoːkaːmɛː
Info: {'phoneme_feature_labels': ['syl', 'son', '

In [9]:
controlled_streams = Register()
for _ in range(N_STREAMS_PER_INPUT):
    for stream in make_streams(
        controlled_lexicons, 
        max_rhythmicity=None, 
        num_repetitions=N_REPS
        ):
        controlled_streams.append(stream)

print_stream_info(controlled_streams[0])

len(controlled_streams)

Stream: boː|hiː|kuː|kaː|foː|nyː|zyː|mɛː|tyː|høː|vaː|ʃoː|nyː|ʃoː|foː|vaː|kuː|hiː|tyː|kaː|høː|mɛː|boː|zyː|foː|høː|boː|kuː|tyː|mɛː|hiː|nyː|kaː|ʃoː|zyː|vaː|zyː|høː|kuː|mɛː|nyː|foː|boː|vaː|kaː|hiː|ʃoː|tyː|nyː|tyː|hiː|høː|foː|zyː|kaː|kuː|vaː|boː|mɛː|ʃoː|vaː|hiː|mɛː|foː|kaː|boː|ʃoː|kuː|nyː|høː|zyː|tyː|kuː|boː|høː|tyː|vaː|mɛː|zyː|nyː|hiː|foː|ʃoː|kaː|mɛː|høː|nyː|vaː|foː|kuː|ʃoː|boː|kaː|tyː|zyː|hiː|kaː|nyː|boː|tyː|foː|mɛː|kuː|zyː|ʃoː|hiː|vaː|høː|kaː|zyː|kuː|foː|tyː|ʃoː|høː|hiː|boː|nyː|mɛː|vaː|nyː|kuː|høː|ʃoː|mɛː|kaː|vaː|tyː|boː|foː|hiː|zyː|boː|kaː|foː|mɛː|høː|zyː|kuː|ʃoː|tyː|hiː|vaː|nyː|zyː|kaː|vaː|hiː|boː|foː|ʃoː|kuː|nyː|tyː|høː|mɛː|kuː|mɛː|vaː|zyː|nyː|ʃoː|kaː|boː|tyː|foː|hiː|høː|nyː|foː|zyː|boː|ʃoː|hiː|mɛː|tyː|kuː|vaː|kaː|høː|boː|nyː|hiː|kaː|zyː|foː|høː|kuː|tyː|vaː|mɛː|ʃoː|zyː|hiː|kuː|høː|tyː|kaː|nyː|boː|mɛː|foː|vaː|ʃoː|mɛː|kaː|tyː|ʃoː|foː|boː|hiː|nyː|vaː|kuː|zyː|høː|ʃoː|høː|foː|tyː|mɛː|nyː|kaː|hiː|zyː|vaː|boː|kuː|hiː|ʃoː|vaː|høː|kaː|mɛː|boː|zyː|tyː|nyː|kuː|foː|kuː|kaː|ʃoː|boː|vaː|foː|nyː|høː|

315

### Random / uncontrolled lexicons (baseline)

In [10]:
random_lexicons = make_lexicons(words, n_lexicons=N_LEXICONS, n_words=N_WORDS_PER_LEXICON, control_features=False)

for i, lex in enumerate(random_lexicons):
    print(i, lex)

print("")
print("Example Lexicon:", random_lexicons[0])
print("Info:", random_lexicons[0].info)
print("")

0 faːɡiːryː|riːfaːkɛː|faːdoːheː|fuːriːkaː
1 roːfuːɡiː|høːtoːfuː|roːfaːkøː|koːvaːlyː
2 vaːtiːhøː|nuːɡiːfaː|huːfoːtɛː|beːzuːhøː
3 riːfaːkøː|kɛːfoːniː|køːvaːnyː|faːniːkuː
4 ɡiːfoːryː|fyːhiːtuː|riːfoːkuː|piːzuːheː
5 beːhoːzuː|ʃeːhoːbøː|fuːriːkaː|ɡiːlyːfoː
6 tiːheːfoː|fiːlaːkøː|ryːkoːfaː|ryːfoːɡɛː
7 niːfoːkaː|ɡiːfaːlyː|kaːfoːniː|tiːheːvaː
8 vaːriːkoː|huːpoːzɛː|heːpoːʃøː|hoːfiːtɛː
9 foːhøːtiː|ɡaːreːfiː|faːdoːheː|faːhøːtuː
10 heːtyːfaː|kuːlyːfoː|tyːfaːhoː|koːnøːvaː
11 ɡiːfoːryː|fuːtyːhiː|foːheːtɛː|huːfoːtiː
12 baːzuːheː|heːtuːfiː|foːdeːhøː|reːkoːfyː
13 fyːraːkɛː|kɛːfaːnyː|piːzuːheː|foːtiːheː
14 koːfiːnøː|ɡaːlyːfuː|tiːheːfoː|fyːriːkoː
15 ɡaːryːfoː|foːtiːheː|ʃiːpoːhøː|vaːniːkuː
16 faːtiːhøː|faːdeːhuː|ɡaːryːfoː|hoːfiːtɛː
17 koːfiːnøː|hoːtyːvaː|foːkuːriː|heːtyːfaː
18 tuːfoːhøː|reːfoːɡyː|luːɡaːfoː|zuːhiːboː
19 ɡiːfoːryː|tyːfaːhoː|koːfiːlyː|fiːkuːnøː
20 reːɡaːfiː|tuːfaːheː|faːtoːhiː|faːhøːtuː

Example Lexicon: faːɡiːryː|riːfaːkɛː|faːdoːheː|fuːriːkaː
Info: {'phoneme_feature_labels': ['syl', 'son', '

In [11]:
random_streams = Register()
for _ in range(N_STREAMS_PER_INPUT):
    for stream in make_streams(
        random_lexicons, 
        max_rhythmicity=None, 
        num_repetitions=N_REPS
        ):
        random_streams.append(stream)
        
print_stream_info(random_streams[0])

len(random_streams)

### Reference lexicons from the literature

In [ ]:
lexicons = [
 [['pi', 'ɾu', 'ta'],
  ['ba', 'ɡo', 'li'],
  ['to', 'ku', 'da'],
  ['ɡu', 'ki', 'bo']],
 [['pa', 'be', 'la'],
  ['di', 'ne', 'ka'],
  ['lu', 'fa', 'ri'],
  ['xi', 'so', 'du']],
 [['ma', 'xu', 'pe'],
  ['xe', 'ro', 'ɡa'],
  ['de', 'mu', 'si'],
  ['fo', 'le', 'ti']],
 [['pu', 'ke', 'mi'],
  ['ra', 'fi', 'nu'],
  ['bi', 'na', 'po'],
  ['me', 'do', 'xi']],
 [['no', 'ni', 'xe'],
  ['bu', 'lo', 'te'],
  ['re', 'mo', 'fu'],
  ['ko', 'tu', 'sa']],
 [['mi', 'lo', 'de'],
  ['da', 'le', 'bu'],
  ['no', 'ru', 'pa'],
  ['ka', 'te', 'xi']],
 [['ne', 'do', 'li'],
  ['ri', 'fo', 'nu'],
  ['ba', 'to', 'ɡu'],
  ['ki', 'ra', 'pu']],
 [['ɡo', 'na', 'be'],
  ['mu', 'di', 'la'],
  ['ro', 'ni', 'xe'],
  ['pi', 'ku', 'sa']],
 [['fu', 'bi', 're'],
  ['xe', 'tu', 'si'],
  ['ta', 'fi', 'ko'],
  ['ke', 'ma', 'po']],
 [['ti', 'fa', 'xu'],
  ['so', 'du', 'xi'],
  ['me', 'lu', 'bo'],
  ['ɡa', 'ni', 'pe']],
 [['mi', 'po', 'la'],
  ['za', 'bɛ', 'tu'],
  ['ʁo', 'ki', 'sɛ'],
  ['nu', 'ɡa', 'di']],
 [['dɛ', 'mʊ', 'ri'],
  ['sɛ', 'ni', 'ɡɛ'],
  ['ræ', 'ku', 'səʊ'],
  ['pi', 'lɛ', 'ru']],
 [['ki', 'fəʊ', 'bu'],
  ['lu', 'fɑ', 'ɡi'],
  ['pæ', 'beɪ', 'lɑ'],
  ['tɑ', 'ɡəʊ', 'fʊ']],
 [['bi', 'du', 'pɛ'],
  ['məʊ', 'bɑ', 'li'],
  ['rɛ', 'ɡæ', 'tʊ'],
  ['sæ', 'tɛ', 'kəʊ']],
 [['bəʊ', 'dɑ', 'mɛ'],
  ['fi', 'nəʊ', 'pɑ'],
  ['ɡʊ', 'rɑ', 'təʊ'],
  ['ləʊ', 'kæ', 'neɪ']],
 [['fɛ', 'si', 'nɑ'],
  ['kɛ', 'su', 'dəʊ'],
  ['mæ', 'pʊ', 'di'],
  ['ti', 'mi', 'nu']],
 [['tu', 'pi', 'ɹoʊ'],
  ['ɡoʊ', 'la', 'bu'],
  ['pa', 'doʊ', 'ti'],
  ['bi', 'da', 'ku']],
 [['meɪ', 'lu', 'ɡi'],
  ['ɹa', 'fi', 'nu'],
  ['pu', 'keɪ', 'mi'],
  ['toʊ', 'na', 'poʊ']],
 [['ɡoʊ', 'la', 'tu'],
  ['da', 'ɹoʊ', 'pi'],
  ['ti', 'bu', 'doʊ'],
  ['pa', 'bi', 'ku']],
 [['poʊ', 'fi', 'mu'],
  ['noʊ', 'vu', 'ka'],
  ['vi', 'koʊ', 'ɡa'],
  ['ba', 'fu', 'ɡi']],
 [['ma', 'nu', 'toʊ'],
  ['ni', 'moʊ', 'lu'],
  ['voʊ', 'ɹi', 'fa'],
  ['li', 'du', 'ɹa']]]

ref_lexicons = list(map(to_lexicon, lexicons))

for i, lex in enumerate(ref_lexicons):
    print(i, lex)

print("")
print("Example Lexicon:", ref_lexicons[0])
print("Info:", ref_lexicons[0].info)
print("")

0 piɾuta|baɡoli|tokuda|ɡukibo
1 pabela|dineka|lufari|xisodu
2 maxupe|xeroɡa|demusi|foleti
3 pukemi|rafinu|binapo|medoxi
4 nonixe|bulote|remofu|kotusa
5 milode|dalebu|norupa|katexi
6 nedoli|rifonu|batoɡu|kirapu
7 ɡonabe|mudila|ronixe|pikusa
8 fubire|xetusi|tafiko|kemapo
9 tifaxu|soduxi|melubo|ɡanipe
10 mipola|zabɛtu|ʁokisɛ|nuɡadi
11 dɛmʊri|sɛniɡɛ|rækusəʊ|pilɛru
12 kifəʊbu|lufɑɡi|pæbeɪlɑ|tɑɡəʊfʊ
13 bidupɛ|məʊbɑli|rɛɡætʊ|sætɛkəʊ
14 bəʊdɑmɛ|finəʊpɑ|ɡʊrɑtəʊ|ləʊkæneɪ
15 fɛsinɑ|kɛsudəʊ|mæpʊdi|timinu
16 tupiɹoʊ|ɡoʊlabu|padoʊti|bidaku
17 meɪluɡi|ɹafinu|pukeɪmi|toʊnapoʊ
18 ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku
19 poʊfimu|noʊvuka|vikoʊɡa|bafuɡi
20 manutoʊ|nimoʊlu|voʊɹifa|liduɹa

Example Lexicon: piɾuta|baɡoli|tokuda|ɡukibo
Info: {'syllable_feature_labels': [['son', 'back', 'hi', 'lab', 'cor', 'cont', 'lat', 'nas', 'voi'], ['back', 'hi', 'lo', 'lab', 'tense', 'long']], 'syllable_type': 'cv', 'cumulative_feature_repetitiveness': 5, 'max_pairwise_feature_repetitiveness': 2}



In [ ]:
ref_streams = Register()
for _ in range(N_STREAMS_PER_INPUT):
    for stream in make_streams(
        ref_lexicons, 
        max_rhythmicity=None, 
        num_repetitions=N_REPS
        ):
        ref_streams.append(stream)

print_stream_info(ref_streams[0])

len(ref_streams)

Stream: ɾu|ku|ba|ɡo|ta|ɡu|ki|pi|to|bo|li|da|ta|ɡo|bo|ku|da|ki|li|to|ɾu|ba|ɡu|pi|da|ɾu|ta|to|ba|li|ɡu|bo|ɡo|pi|ki|ku|to|ku|ki|ta|li|pi|ɾu|ɡo|ɡu|ba|bo|da|bo|pi|ɡo|ku|ta|ɾu|da|ba|to|ki|ɡu|li|ɡo|to|ta|pi|bo|ba|da|ɡu|ku|li|ki|ɾu|ɡu|da|ɡo|li|ɾu|bo|ta|ba|ki|to|pi|ku|pi|li|ba|ɾu|to|da|ku|ɡo|ki|bo|ɡu|ta|ki|da|li|bo|ɾu|pi|ɡu|to|ɡo|ba|ta|ku|ɡu|ɡo|ɾu|li|ku|bo|ki|ba|pi|ta|da|to|li|ta|bo|to|ɡu|ɾu|ki|ɡo|da|pi|ba|ku|ɾu|ki|ɡu|ɡo|ku|ta|ba|li|to|da|pi|bo|ɡo|bo|li|ku|ɾu|ɡu|ta|ki|ba|to|pi|da|ku|bo|pi|ɡo|da|ta|ɾu|li|ba|ɡu|ki|to|ba|ku|ɡo|pi|ɾu|ta|bo|to|ɡu|da|ki|li|bo|ta|li|ɡo|ba|ki|da|ɾu|to|ku|pi|ɡu|pi|ta|da|ba|ɡo|ɾu|ku|li|ki|bo|ɡu|to|ki|ku|ba|ta|to|bo|ɾu|da|ɡo|ɡu|li|pi|li|ta|ku|to|ɡo|ki|pi|ba|bo|da|ɡu|ɾu|ba|da|li|ɡu|bo|ku|ki|ɾu|ɡo|ta|pi|to|li|da|bo|ba|ɾu|pi|ki|ɡo|to|ta|ɡu|ku|ɡu|ba|pi|ku|da|to|ɾu|bo|ki|ta|ɡo|li|ɾu|ku|ɡu|to|ki|ba|ɡo|pi|da|bo|ta|li|da|to|ta|pi|ki|ɾu|li|bo|ba|ɡu|ku|ɡo|to|li|ta|ɡo|bo|ɡu|ki|ku|da|ba|ɾu|pi|to|pi|ba|bo|li|ku|ɾu|ki|ɡo|da|ta|ɡu|pi|ɡu|ta|ku|to|da|ɡo|ba|ki|li|ɾu|bo|pi|ku|ta|to|ɡu|ɾu|ba

315

### Collect Results

We collect all stream generation results and their feature repetitiveness scores in a dataframe.

In [ ]:
import pandas as pd

data = {"Control": [], "Lexicon": [], "Feature": [], "PRI": [], "Stream TP mode": [], "Stream": []}

mode_to_mode = {  # TP-random position-random; TP-random position-fixed and TP-structured
    "random": "TP-random position-random",
    "word_structured": "TP-structured",
    "position_controlled": "TP-random position-fixed"
}

for control, streams in [("Controlled lexicons (ARC)", controlled_streams), ("Reference lexicons (Literature)", ref_streams), ("Random lexicons (Baseline)", random_streams)]:
    for stream in streams:
        for k, v in stream.info["rhythmicity_indexes"].items():
            data["Feature"].append(k)
            data["PRI"].append(v)
            data["Control"].append(control)
            data["Lexicon"].append(str(stream.info["lexicon"]))
            data["Stream TP mode"].append(mode_to_mode[stream.info["stream_tp_mode"]])
            data["Stream"].append("|".join(syll.id for syll in stream))
        data["Feature"].append("max")
        data["PRI"].append(max(stream.info["rhythmicity_indexes"].values()))
        data["Control"].append(control)
        data["Lexicon"].append(str(stream.info["lexicon"]))
        data["Stream TP mode"].append(mode_to_mode[stream.info["stream_tp_mode"]])
        data["Stream"].append("|".join(syll.id for syll in stream))

df = pd.DataFrame(data).sort_values(["Control", "Lexicon", "Stream TP mode"]).reset_index(drop=True)

import os
os.makedirs("results/", exist_ok=True)
df.to_csv("results/full_dataset.csv")

df

,Control,Lexicon,Feature,PRI,Stream TP mode,Stream
0,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː,phon_1_son,0.058824,TP-random position-fixed,faː|foː|ɡyː|reː|koː|heː|byː|tɛː|mɛː|ʃuː|hoː|zu...
1,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː,phon_1_back,0.011204,TP-random position-fixed,faː|foː|ɡyː|reː|koː|heː|byː|tɛː|mɛː|ʃuː|hoː|zu...
2,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː,phon_1_hi,0.011204,TP-random position-fixed,faː|foː|ɡyː|reː|koː|heː|byː|tɛː|mɛː|ʃuː|hoː|zu...
3,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː,phon_1_lab,0.058824,TP-random position-fixed,faː|foː|ɡyː|reː|koː|heː|byː|tɛː|mɛː|ʃuː|hoː|zu...
4,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː,phon_1_cor,0.051821,TP-random position-fixed,faː|foː|ɡyː|reː|koː|heː|byː|tɛː|mɛː|ʃuː|hoː|zu...
...,...,...,...,...,...,...
15115,Reference lexicons (Literature),ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,phon_2_lo,0.148459,TP-structured,ɡoʊ|la|tu|ti|bu|doʊ|da|ɹoʊ|pi|pa|bi|ku|ɡoʊ|la|...
15116,Reference lexicons (Literature),ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,phon_2_lab,0.142857,TP-structured,ɡoʊ|la|tu|ti|bu|doʊ|da|ɹoʊ|pi|pa|bi|ku|ɡoʊ|la|...
15117,Reference lexicons (Literature),ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,phon_2_tense,0.000000,TP-structured,ɡoʊ|la|tu|ti|bu|doʊ|da|ɹoʊ|pi|pa|bi|ku|ɡoʊ|la|...
15118,Reference lexicons (Literature),ɡoʊlatu|daɹoʊpi|tibudoʊ|pabiku,phon_2_long,0.000000,TP-structured,ɡoʊ|la|tu|ti|bu|doʊ|da|ɹoʊ|pi|pa|bi|ku|ɡoʊ|la|...


In [ ]:
# There is always randomness in the generation of the lexicons etc., so if you want the exact data from the publication uncomment below 
# df = pd.read_csv("data_submission/full_dataset.csv")

df_lexicons = df[["Control", "Lexicon"]].drop_duplicates().reset_index(drop=True)
df_lexicons.to_csv("results/all_lexicons.csv")
df_lexicons

,Control,Lexicon
0,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|faːtɛːheː|reːfoːɡyː
1,Controlled lexicons (ARC),byːhoːzuː|ʃuːkoːmɛː|høːdeːfaː|reːfoːɡyː
2,Controlled lexicons (ARC),faːhoːtɛː|riːkaːfyː|ʃoːpeːhøː|meːzyːkoː
3,Controlled lexicons (ARC),faːhoːtɛː|roːkuːfyː|ʃuːbiːheː|ɡiːʃoːmyː
4,Controlled lexicons (ARC),fyːkoːruː|høːdeːfaː|tyːhoːfuː|ryːfoːɡiː
...,...,...
58,Reference lexicons (Literature),pukemi|rafinu|binapo|medoxi
59,Reference lexicons (Literature),tifaxu|soduxi|melubo|ɡanipe
60,Reference lexicons (Literature),tupiɹoʊ|ɡoʊlabu|padoʊti|bidaku
61,Reference lexicons (Literature),ɡonabe|mudila|ronixe|pikusa


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from pingouin import ttest

# There is always randomness in the generation of the lexicons etc., so if you want the exact data from the publication uncomment below 
# df = pd.read_csv("data_submission/full_dataset.csv")

tp_modes_pretty = ["TP-random position-random", "TP-random position-fixed", "TP-structured"]
dfs = []

for i, tp_mode in enumerate(tp_modes_pretty):
    df2 = df[(df["Stream TP mode"] == tp_mode) & (df["Feature"] == "max")]
    cat1 = df2[df2['Control']=='Controlled lexicons (ARC)']["PRI"]
    cat2 = df2[df2['Control']=='Reference lexicons (Literature)']["PRI"]
    this = ttest(list(cat1), list(cat2), alternative="less")
    this.index = [f"controlled vs. reference {tp_mode}"]
    dfs.append(this)

print("")

for i, tp_mode in enumerate(tp_modes_pretty):
    df2 = df[(df["Stream TP mode"] == tp_mode) & (df["Feature"] == "max")]
    cat1 = df2[df2['Control']=='Controlled lexicons (ARC)']["PRI"]
    cat2 = df2[df2['Control']=='Random lexicons (Baseline)']["PRI"]
    this = ttest(list(cat1), list(cat2), alternative="less")
    this.index = [f"controlled vs. random baseline {tp_mode}"]
    dfs.append(this)

print("")

for i, tp_mode in enumerate(tp_modes_pretty):
    df2 = df[(df["Stream TP mode"] == tp_mode) & (df["Feature"] == "max")]
    cat1 = df2[df2['Control']=='Reference lexicons (Literature)']["PRI"]
    cat2 = df2[df2['Control']=='Random lexicons (Baseline)']["PRI"]
    this = ttest(list(cat1), list(cat2), alternative="less")
    this.index = [f"reference vs. random baseline {tp_mode}"]
    dfs.append(this)

ttest_df = pd.concat(dfs)

display(ttest_df)

ttest_df.to_csv("results/ttest_results.csv")

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
controlled vs. reference TP-random position-random,4.713530,208,less,9.999978e-01,"[-inf, 0.01]",0.650529,0.0,1.125394e-10
controlled vs. reference TP-random position-fixed,-10.011785,208,less,8.238282e-20,"[-inf, -0.08]",1.381758,4.673e+16,1.000000e+00
controlled vs. reference TP-structured,-7.973772,208,less,5.048530e-14,"[-inf, -0.08]",1.100485,1.184e+11,1.000000e+00
controlled vs. random baseline TP-random position-random,0.180549,208,less,5.715513e-01,"[-inf, 0.0]",0.024918,0.305,3.401436e-02
controlled vs. random baseline TP-random position-fixed,-9.361886,208,less,6.592708e-18,"[-inf, -0.22]",1.292063,6.649e+14,1.000000e+00
controlled vs. random baseline TP-structured,-13.470629,208,less,1.886071e-30,"[-inf, -0.29]",1.859123,1.167e+27,1.000000e+00
reference vs. random baseline TP-random position-random,-4.206095,208,less,1.930524e-05,"[-inf, -0.0]",0.580496,953.452,9.945752e-01
reference vs. random baseline TP-random position-fixed,-5.430899,208,less,7.786872e-08,"[-inf, -0.11]",0.749535,1.559e+05,9.999178e-01
reference vs. random baseline TP-structured,-8.222437,208,less,1.067860e-14,"[-inf, -0.18]",1.134804,5.276e+11,1.000000e+00


## Example ARC Lexicon From Appendix

In [ ]:
example_arc_lexicon = to_lexicon([["heː", "doː", "faː"], ["riː", "foː", "ɡyː"], ["ʃuː", "hiː", "boː"], ["vaː", "kuː", "niː"]])
print("Example Lexicon:", example_arc_lexicon)

streams = make_streams(
        [example_arc_lexicon], 
        max_rhythmicity=None, 
        num_repetitions=N_REPS
        )

for stream in streams:
        print_stream_info(stream)

Example Lexicon: heːdoːfaː|riːfoːɡyː|ʃuːhiːboː|vaːkuːniː
Stream: faː|kuː|heː|hiː|ʃuː|ɡyː|foː|boː|niː|riː|vaː|doː|ʃuː|doː|faː|boː|riː|hiː|niː|ɡyː|kuː|vaː|heː|foː|kuː|niː|vaː|ɡyː|faː|riː|doː|hiː|boː|heː|ʃuː|foː|riː|foː|ʃuː|hiː|faː|heː|doː|kuː|ɡyː|boː|vaː|niː|faː|foː|vaː|kuː|doː|boː|ʃuː|niː|hiː|ɡyː|heː|riː|heː|boː|doː|foː|hiː|kuː|faː|ɡyː|ʃuː|vaː|riː|niː|kuː|foː|faː|niː|heː|ɡyː|doː|riː|ʃuː|boː|hiː|vaː|ʃuː|kuː|boː|foː|doː|heː|vaː|faː|hiː|riː|ɡyː|niː|boː|faː|doː|ɡyː|vaː|foː|heː|niː|ʃuː|riː|kuː|hiː|heː|faː|vaː|hiː|doː|niː|foː|ɡyː|riː|boː|kuː|ʃuː|faː|ʃuː|heː|kuː|riː|foː|niː|doː|vaː|boː|ɡyː|hiː|foː|boː|faː|doː|vaː|kuː|ʃuː|niː|ɡyː|hiː|heː|riː|faː|foː|doː|ɡyː|boː|kuː|niː|hiː|riː|vaː|ʃuː|heː|doː|foː|faː|vaː|niː|boː|hiː|ʃuː|ɡyː|heː|kuː|riː|niː|heː|boː|ɡyː|riː|doː|ʃuː|hiː|foː|kuː|vaː|faː|riː|boː|foː|hiː|doː|kuː|ɡyː|vaː|heː|niː|faː|ʃuː|riː|hiː|niː|doː|heː|ɡyː|foː|ʃuː|faː|kuː|boː|vaː|ɡyː|niː|kuː|heː|foː|vaː|doː|riː|faː|hiː|boː|ʃuː|vaː|hiː|kuː|faː|ɡyː|doː|niː|foː|riː|ʃuː|boː|heː|faː|heː|hiː|vaː|riː|ɡyː